In [ ]:
#这只是申请表，对逾期表之类的其他表也可以做相同的操作，然后就可以得到每个月的逾期，还款，申请，用款情况
# DAYS_DECISION为申请时间距离当前申请时刻的时间间隔，将其转为月份
df_pre_application['appl_month'] = df_pre_application.DAYS_DECISION.apply(lambda x: int(x/30))
# NAME_CONTRACT_TYPE为标的的类型，消费贷，现金贷，先进行onehot，后面求和得到每月各类型申请的次数
df_pre_application = pd.concat([df_pre_application, pd.get_dummies(df_pre_application.NAME_CONTRACT_TYPE, prefix='pre_appl_type_cnt')], axis=1)
# NAME_CONTRACT_STATUS为标的状态，与通过，拒绝，有额未使用等，同上
df_pre_application = pd.concat([df_pre_application, pd.get_dummies(df_pre_application.NAME_CONTRACT_STATUS, prefix='pre_appl_type_status_cnt')], axis=1)

# 按月聚合：NAME_CONTRACT_TYPE
df_pre_type_sum_all = df_pre_application.groupby(['SK_ID_CURR', 'appl_month']).agg({'pre_appl_type_cnt_Cash loans': np.sum,
'pre_appl_type_cnt_Consumer loans': np.sum,
'pre_appl_type_cnt_Revolving loans':np.sum})
df_pre_type_sum_all.reset_index(inplace=True)
# 按月聚合：NAME_CONTRACT_STATUS
df_pre_status_sum_all = df_pre_application.groupby([c]).agg({'pre_appl_type_status_cnt_Approved':np.sum,
'pre_appl_type_status_cnt_Canceled':np.sum,
'pre_appl_type_status_cnt_Refused':np.sum})
df_pre_status_sum_all.reset_index(inplace=True)

# 按月聚合： 对申请的金额，授信的额度，每月还款金额进行sum
agg_dict = {'AMT_ANNUITY': np.sum,'AMT_APPLICATION': np.sum,'AMT_CREDIT': np.sum}
df_pre_amt_sum_all = df_pre_application.groupby(['SK_ID_CURR', 'appl_month']).agg(agg_dict)
df_pre_amt_sum_all.columns = ['pre_appl_month_sum_' + f for f in df_pre_amt_sum_all.columns.tolist()]
df_pre_amt_sum_all.reset_index(inplace=True)

# 按月聚合： 每月申请的总次数
df_pre_app_cnt = df_pre_application.groupby(['SK_ID_CURR', 'appl_month']).agg({'SK_ID_PREV': np.size})
df_pre_app_cnt.rename(columns={'SK_ID_PREV': 'pre_appl_cnt'}, inplace=True)
df_pre_app_cnt.reset_index(inplace=True)

# 合并上述信息，即可得到历史申请记录在每月的状态
df_pre_app_combined = pd.merge(df_pre_amt_sum_all, df_pre_type_sum_all, on=['SK_ID_CURR', 'appl_month'], how='inner')
df_pre_app_combined = pd.merge(df_pre_app_combined, df_pre_status_sum_all, on=['SK_ID_CURR', 'appl_month'], how='inner')
df_pre_app_combined = pd.merge(df_pre_app_combined, df_pre_app_cnt, on=['SK_ID_CURR', 'appl_month'], how='inner')

In [ ]:
#之后可以把这些时序信息全部聚合起来建立模型